# ProjectPegasus

Exam project for the Artificial Intelligence Fundamentals course, a.y. 2023/2024

Notebook and code by the NetRiders team: Giordano Scerra, Andrea Marino, Yuri Ermes Negri, Davide Borghini, Davide Marchi

---

## Introduction

This project focuses on the task of riding a pony. To do so, the agent:
- Picks up some carrots
- Feeds the pony with carrots, to pacify him and increase his tameness
- Looks for a saddle
- Applies the saddle on the pony and rides it

## Methodologies

Our code is structured in three main files:
- **Map.py**: These class methods are used for the interactions with the environment: extracting information from the cells, rendering the level and making a proper step in the game (i.e. applying the actions).
- **Agent.py**: Interacts with Map.py and the knowledge base with two methods: `percept()` and `act()`. 
    - `percept()`: reads information from the environment - using attributes from Map - and populates the knowledge base, using methods from the KBwrapper.
    - `act()`: Queries the knowledge base for the best subtask to execute and launch the related methods.
- **KBwrapper.py**: For the agent, its knowledge base is an instance of this class. Based on pyswip library, its methods are used to manage the "kb.pl" Prolog knowledge base by asserting, retracting and querying the appropriate predicates. 

Our project is related with the course for these key topics:
- A* search algorithm
- Knowledge Base
- explore di davideb?

### Subtasks and Interrupts

We decided to implement an intelligent system for decision making based on interruptions. 
In the case the agent has determined that the best action to perform is to undertake a certain subtask, an interruption notifies to the agent that the premises to continue that specific subtask are no longer valid and so a subtask switch is needed. 
So, after every step the agent does in the environment, the premises for the interruption of the current subtask are checked, and if they are satisfied, we interrupt the flow of the actions and ask to the knowledge base what is the best thing to do in that moment. 
The different subtasks for the agent are:
- getCarrot : to go and collect the nearest carrot
- getSaddle : to go and collect the saddle for the pony
- feedSteed : to feed the carrots to the pony 
- applySaddle : to apply the saddle to the pony
- rideSteed : to attempt to ride the pony
- explore : to explore unseen zones of the map

### Knowledge Base

As discussed during the oral presentation, and as described in the previous paragraph, the knowledge base is where both the information is stored and the decision making process for the agent takes place. This entity itself is divided in two parts, in two different languages: Prolog and Python. In facts, Python is used as a wrapper interface to control the Prolog part, thanks to the open source library pyswip. 
The subtasks in the knowledge base are defined in order of relevance. We have:
- getCarrot
- feedSteed
- getSaddle
- applySaddle
- explore (this subtask is chosen when no premise is satisfied for another subtask to start)

After the subtask's section, in our code we have the interruption's section. Here we can find, for every subtask, its own interruption clause.
Then we have some facts, like the pony's tameness assigned to 20, or the definition of a steed.
Eventually, a section where we store some observations like the number of carrots or saddles in our possession, or the pony's actual tameness. 

## Assessment

## Conclusions

## Appendix